In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
dataset = load_iris()
dir(dataset)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [3]:
from numpy.random import permutation

shuffled_idx = permutation(range(dataset.data.shape[0]))

In [4]:
dataset.data = dataset.data[shuffled_idx]
dataset.target = dataset.target[shuffled_idx]

In [222]:
X = dataset.data
y = dataset.target

In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y = ohe.fit_transform(dataset.target.reshape(-1,1))

/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [223]:
X = X/(X.max(axis=0))

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X,y,test_size=0.2)

In [129]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(3,),activation='relu')

In [131]:
mlp.fit(x_train,y_train)

/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [140]:
mlp.predict(x_val[25].reshape(1,-1))

array([2])

In [141]:
y_val[25]

2

In [238]:
from sklearn.base import BaseEstimator

class littleNN(BaseEstimator):
    def __init__(self,num_hidden):
        self.num_hidden = num_hidden
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoidPrime(self,x):
        return x*(1-x)
    
    def correct_weights(self,hidden_layer,output_layer,y_true,silent=0):
              
        if not silent:
            E = np.sum((output_layer-y_true)**2)
            #print('Error: {0}'.format(E))
        
        delta2 = (output_layer-y_true)*self.sigmoidPrime(output_layer)
        self.w2 -= hidden_layer.T.dot(delta2)
        
        delta1 = delta2.dot(self.w2[:-1].T)*self.sigmoidPrime(hidden_layer[:,:-1])
        self.w1 -= self.input_layer.T.dot(delta1) #
        
        
    def fit(self,X,y,num_batches=1,epochs=100):

        m = X.shape[0]
        num_input = X.shape[1]
        num_output = y.shape[1]
       
        samples_per_batch = m//num_batches
        
        self.w1 = 2*np.random.rand(num_input+1,self.num_hidden)-1
        self.w2 = 2*np.random.rand(self.num_hidden+1,num_output)-1
        
        for i in range(epochs):
            #print('Epoch {0}'.format(i))
            k = 0
            for sample, y_true in zip(X,y): #zip(X[k:k+samples_per_batch,:], y[k:k+samples_per_batch,:]):
                
                self.input_layer = np.r_[sample,1].reshape(1,-1)
                hidden_layer = self.sigmoid((np.c_[self.input_layer.dot(self.w1),1].reshape(1,-1)))
                output_layer = self.sigmoid(hidden_layer.dot(self.w2))
                
                '''print(self.input_layer.shape)
                print(self.w1.shape)
                print(hidden_layer.shape)
                print(self.w2.shape)
                print(output_layer.shape)
                print(y_true.shape)'''
                
                self.correct_weights(hidden_layer,output_layer,y_true.reshape(1,-1))
                
                #k += samples_per_batch
    def predict(self,X):
        res = np.zeros((X.shape[0],10))
        
        for i,sample in enumerate(X):
            self.input_layer = np.r_[sample,1].reshape(1,-1)
            hidden_layer = self.sigmoid((np.c_[self.input_layer.dot(self.w1),1].reshape(1,-1)))
            output_layer = self.sigmoid(hidden_layer.dot(self.w2))
            res[i] = np.round(output_layer)
            #print(res[i])
        
        return res

In [220]:
nn = littleNN(10)
cross_val_score(nn,X,y,cv=5,scoring='accuracy')

[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]

array([0.9       , 1.        , 1.        , 1.        , 0.96666667])

In [224]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cross_val_score(lr,X,y,cv=5,scoring='accuracy')

/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/user/PythonInterprener/v

array([0.76666667, 0.83333333, 0.86666667, 0.9       , 0.9       ])

In [218]:
nn = littleNN(3)
nn.fit(x_train,y_train)
res = nn.predict(x_val)

c = 0
for i in range(res.shape[0]):
    if(all(res[i]==y_val[i])):
        c+=1

c/res.shape[0]

[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 0. 1.]


1.0

In [217]:
np.array([1,2,3,4])==np.array([1,2,33,4])

array([ True,  True, False,  True])

In [194]:
np.round(0.7429,decimals=1)

0.7

In [198]:
y

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0

In [207]:
a = np.random.rand(3,4)
a

array([[0.68490815, 0.34709096, 0.74357104, 0.00788877],
       [0.32005102, 0.35725852, 0.89562805, 0.43805302],
       [0.69337965, 0.12506107, 0.4397513 , 0.38414352]])

In [208]:
a[0] = [1,2,3,4]

In [209]:
a

array([[1.        , 2.        , 3.        , 4.        ],
       [0.32005102, 0.35725852, 0.89562805, 0.43805302],
       [0.69337965, 0.12506107, 0.4397513 , 0.38414352]])

In [210]:
for i,sample in enumerate(X):
    print(i,sample)

0 [0.81012658 0.63636364 0.8115942  0.88      ]
1 [0.60759494 0.77272727 0.27536232 0.08      ]
2 [0.93670886 0.63636364 0.88405797 0.76      ]
3 [1.         0.86363636 0.92753623 0.8       ]
4 [0.63291139 0.79545455 0.1884058  0.12      ]
5 [0.69620253 0.95454545 0.20289855 0.08      ]
6 [0.81012658 0.72727273 0.76811594 0.92      ]
7 [0.87341772 0.70454545 0.71014493 0.6       ]
8 [0.81012658 0.72727273 0.65217391 0.6       ]
9 [0.60759494 0.68181818 0.20289855 0.04      ]
10 [0.63291139 0.77272727 0.23188406 0.16      ]
11 [0.82278481 0.63636364 0.66666667 0.6       ]
12 [0.75949367 0.5        0.57971014 0.4       ]
13 [0.74683544 0.72727273 0.69565217 0.72      ]
14 [0.69620253 0.52272727 0.57971014 0.52      ]
15 [0.65822785 0.79545455 0.2173913  0.08      ]
16 [0.70886076 0.68181818 0.65217391 0.6       ]
17 [0.64556962 0.86363636 0.2173913  0.12      ]
18 [0.63291139 0.68181818 0.23188406 0.08      ]
19 [0.74683544 0.68181818 0.73913043 0.72      ]
20 [0.79746835 0.56818182 0.71

In [225]:
import pandas as pd

train_dig = pd.read_csv('digit-recognizer/train.csv')

In [227]:
y = train_dig.pop('label')

In [228]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y = ohe.fit_transform(np.array(y).reshape(-1,1))

/home/user/PythonInterprener/venv/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [229]:
train_dig = train_dig/255

In [231]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_dig,y,test_size=0.2)

In [245]:
nn = littleNN(50)
nn.fit(x_train.values,y_train,epochs=100)

In [246]:
res = nn.predict(x_val.values)

c = 0
for i in range(res.shape[0]):
    if(all(res[i]==y_val[i])):
        c+=1

c/res.shape[0]

0.9188095238095239

In [247]:
nn.w1

array([[-3.83006340e-02, -7.11482790e-01, -6.43894464e-01, ...,
         7.41371401e-01, -9.96892926e-01,  5.91131050e-02],
       [-4.74998079e-02, -5.79528564e-01,  6.85188434e-01, ...,
         1.77600649e-02,  9.09006657e-01,  7.38658106e-01],
       [-2.65790629e-01,  1.77104299e-01, -6.75617219e-01, ...,
         2.72290779e-01, -5.68197571e-01, -8.72056400e-01],
       ...,
       [ 1.11264375e-01, -4.15272516e-01, -6.82988075e-01, ...,
         4.17429874e-01, -6.13697200e-01, -8.71148716e-01],
       [ 4.01230085e-01, -2.13513531e-01,  9.57990630e-01, ...,
        -5.43729585e-01,  6.87360689e-03, -5.09384329e-01],
       [ 6.56590259e-01, -3.07171859e+00,  4.07859302e+00, ...,
         9.31639699e+00,  1.22417041e+01, -8.28426765e+00]])